<img src="http://www.ubu.es/sites/default/files/portal_page/images/logo_color_2l_dcha.jpg" height="150" width="150" align="right"/>

## Collaborative Filtering (2)
[Nacho Santos](www.nacho.santos.name)

## Import python packages

In [59]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [60]:
# Other functions necessary for this assignment
# The python file "recommender_system.py" must be in the same folder as this notebook, otherwise,
# you have to add the path to the file
from recommender_system import *

# Ruta al archivo 'u.data' en tu carpeta de trabajo de Jupyter
file_path_ratings = 'u.data'

# Leer el archivo 'u.data' y crear las matrices "Y" y "R"
ratings_data = np.genfromtxt(file_path_ratings, delimiter='\t', dtype=int)
num_users = np.max(ratings_data[:, 0])
num_movies = np.max(ratings_data[:, 1])

dim_Y = (num_users, num_movies)
dim_R = (num_users, num_movies)

# Supongamos que estas son las matrices cargadas
Y = np.load('matriz_Y.npy')
R = np.load('matriz_R.npy')

# Verificar las dimensiones de las matrices Y y R
if Y.shape == dim_Y and R.shape == dim_R:
    print("Las dimensiones son correctas:")
    print(f"Dimensiones de Y: {Y.shape} - Esperadas: {dim_Y}")
    print(f"Dimensiones de R: {R.shape} - Esperadas: {dim_R}")
else:
    print("Error en las dimensiones:")
    print(f"Dimensiones de Y: {Y.shape} - Esperadas: {dim_Y}")
    print(f"Dimensiones de R: {R.shape} - Esperadas: {dim_R}")
    
# Imprimir una muestra de las matrices Y y R
print("Matriz Y:")
print(Y[:5, :5])  # Imprime las primeras 5 filas y 5 columnas de Y

print("\nMatriz R:")
print(R[:5, :5])  # Imprime las primeras 5 filas y 5 columnas de R


Error en las dimensiones:
Dimensiones de Y: (1682, 943) - Esperadas: (943, 1682)
Dimensiones de R: (1682, 943) - Esperadas: (943, 1682)
Matriz Y:
[[5. 4. 0. 0. 4.]
 [3. 0. 0. 0. 3.]
 [4. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]]

Matriz R:
[[1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [61]:
# This line is necessary to show matplotlib plots inside the jupyter notebook
%matplotlib inline

## 2 The cost funcion J and the gradient of J
The objective of this point is to build a function to compute the cost J and the corresponding gradient of J. In particular, you are going to implement a function called **cofiCostFunc()** with the arguments(inputs) and outputs detailed below (the code of the function is partially predefined in a cell right after).

**Arguments** (in this order)
* *parameters* (the paramaters of the cost function, i.e. X and $\theta$
* *Y* (matrix of ratings)
* *R* (matrix of watched movies)
* *n_users* (number of users)
* *n_movies* (number of movies)
* *n_characteristics* (number of the filter characteristics)
* *landa* (regularization parameter)

**Outputs** (in this order)
* *cost* (value of the cost function J)
* *gradient* (gradient of the cost function J)
* La función cofiCostFunc que proporcioné anteriormente encapsula la fórmula en un bloque de código en Python. La fórmula se implementa paso a paso dentro de la función. A continuación, te muestro cómo cada parte de la fórmula se traduce en el código:

    Error Cuadrático:
        predictions = X.dot(Theta.T) calcula todas las predicciones de las calificaciones como el producto punto entre las características de las películas XX y las preferencias del usuario ΘΘ.
        errors = (predictions - Y) * R calcula la diferencia entre las predicciones y las calificaciones reales YY, pero solo para los elementos donde RR es igual a 1 (es decir, donde una calificación existe).

    Costo sin Regularización:
        cost = (1/2) * np.sum(errors**2) calcula la mitad de la suma de los errores cuadráticos, que es la primera parte de la fórmula.

    Regularización:
        cost += (lambda_reg/2) * (np.sum(Theta**2) + np.sum(X**2)) añade la regularización para ΘΘ y XX al costo, que es la suma de la mitad del término de regularización multiplicado por la suma de los cuadrados de los elementos de ΘΘ y XX.

    Gradientes:
        X_grad = errors.dot(Theta) + lambda_reg * X calcula el gradiente para XX con regularización.
        Theta_grad = errors.T.dot(X) + lambda_reg * Theta calcula el gradiente para ΘΘ con regularización.

    Vector de Gradientes:
        gradient = np.concatenate([X_grad.ravel(), Theta_grad.ravel()]) aplana y concatena los gradientes de XX y ΘΘ en un solo vector para su uso en algoritmos de optimización.
  

In [62]:
# Cost and gradient function
def cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, lamb):
    # parameters: vector with the matrices X and Theta folded
    # Y: matrix of ratings
    # R: matrix of watched movies
    # n_users: number of users (number of columns of the matrix Y)
    # n_movies: number of movies (number of rows of the matrix Y)
    # n_features: number of movies' features (a parameter of the CF algorithm)
    # lamb: regularization term
    #
    # cofiCostFunc returns the cost J and the gradient of J

    # You need to return the following values correctly
    cost = 0
    gradient = np.zeros_like(parameters)
    
    # Desplegar X y Theta de parameters
    X = parameters[:n_movies * n_features].reshape(n_movies, n_features)
    Theta = parameters[n_movies * n_features:].reshape(n_users, n_features)
    
    # Calcular las predicciones de las calificaciones
    predictions = X.dot(Theta.T)
    
    # Calcular los errores solo para las películas que han sido calificadas
    errors = (predictions - Y) * R
    
    # Calcular el coste con regularización
    cost = 0.5 * np.sum(errors ** 2) + (lamb / 2) * (np.sum(Theta ** 2) + np.sum(X ** 2))
    
    # Calcular los gradientes con regularización
    X_grad = errors.dot(Theta) + lamb * X
    Theta_grad = errors.T.dot(X) + lamb * Theta
    
    # Replegar los gradientes en un solo vector
    gradient = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    
    return cost, gradient

### 2.1 Input *parameters* of the cofiCostFunc

**Features X and preferences Theta**

The Collaborative Filtering (CF) algorithm is based on two sets of lineas regressions, the first one corresponds to the movies' features X, and the second one corresponds to the users' preferences Theta. Assuming n features, the matrix X will be:

$$X=\begin{bmatrix}x^{(1)}_{1} & ...& x^{(1)}_{n} \\. & ...& .\\x^{(m)}_{1} & ...& x^{(m)}_{n} \end{bmatrix}$$

where the i-th row of X corresponds to the feature vector $x^{(i)}$ for the i-th movie.

And the matrix Theta will be:

$$Theta=\Theta=\begin{bmatrix}\theta^{(1)}_{1} & ...& \theta^{(1)}_{n} \\. & ...& .\\\theta^{(u)}_{1} & ...& \theta^{(u)}_{n} \end{bmatrix}$$

where the j-th row of Theta corresponds to the preference vector $\theta^{(j)}$ for the j-th user. 

**Passing X and Theta to cofiCostFunc**

We are going to use a optimize package scipy.optimize that requires using a **flattened vector** of parameters. However, in our problem tha parameters to be optimized are represented by two matrices, i.e. X and Theta. So, X and Theta must be passed to the cofiCostFunc as a **(mxn)+(u+n) vector**, called **parameters**:

$${ \left[ \begin{matrix} { x }^{ (1) }, & ... & { x }^{ (m) }, \end{matrix}\begin{matrix} \theta ^{ (1) }, & ... & \theta ^{ (u) } \end{matrix} \right]  }_{ (m\cdot n)+(u\cdot n) }$$ 

However, inside the function, you can unfold the vector **parameters** and build the matrices X and Theta to compute J and the gradients according to the equations explained in class.

### 2.2 Computing the cost J
Suppose that the vector of features $x^{(i)}$ of the film i and the vector of preferences $\theta^{(j)}$ of the user j are known, then the **estimate of the rating** of the user j for the movie i will be:

$$\widehat{y}^{(i,j)}=x^{(i)}(\theta^{(j)})^{T}$$

The error of the estimate will be the difference between the estimate of rating $\widehat{y}^{(i,j)}$ and the real ratings $y^{(i,j)}$

The **cost J** is defined as the the average of the squares of the errors plus two regularization terms:

$$J=\frac { 1 }{ 2 } \sum _{ (i,j):r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) ^{ 2 } } +\quad \frac { \lambda  }{ 2 } \sum _{ i=1 }^{ m }{ \sum _{ k=1 }^{ n }{ ({ x }_{ k }^{ (i) })^{ 2 } }  } +\frac { \lambda  }{ 2 } \sum _{ j=1 }^{ u }{ \sum _{ k=1 }^{ n }{ ({ \theta  }_{ k }^{ (j) })^{ 2 } }  } $$


### Task 7
***
Implement the cost J as a vectorized expression (recommended). For example, the estimate of ratings can be expressed as:

$$\widehat{Y}=X\Theta^{T}$$

Now, go back and **complete the cofiCostFunc code to compute the cost J**. Remeber that J is scalar value.

### 2.3 Checking the cost J
Now, you will import a data set and check the cofiCostFunc.

In [63]:
# load dataset for checking
Y=np.load('YmatrixTest.npy')
R=np.load('RmatrixTest.npy')
X=np.load('XmatrixTest.npy')
Theta=np.load('ThetamatrixTest.npy')

# dimension
n_users = Y.shape[1]
n_movies = Y.shape[0]
n_features = X.shape[1]

# Aplanar las matrices X y Theta para crear el vector de parámetros
parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Definir el parámetro de regularización
# Debes reemplazar esto con el valor correcto de lambda que estás utilizando en tu configuración
lambda_reg = 22.22

# Calcular el costo usando la función cofiCostFunc
cost, _ = cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, lambda_reg)

# Mostrar el costo
print('Coste: ', cost)


Coste:  157.31409370104396


### Task 8
***
Call cofiCostFunc with lamb=0 (without regularization term) and check the result

In [64]:
# Evaluate Cost J (without regularization term)
J=0
parameters=[]

# YOUR CODE ..................................
# call cofiCostFunc with lamb=0 (without regularization term)

# Aplanar las matrices X y Theta para crear el vector de parámetros
parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Llamar a cofiCostFunc con lamb = 0 (sin término de regularización)
J, _ = cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, lamb=0)

# YOUR CODE (end)..................................

print('The value of J (without regularization term) is %0.2f (it should be 22.22)' % J )

The value of J (without regularization term) is 22.22 (it should be 22.22)


### Task 9
***
Call cofiCostFunc with lamb=1.5 (with regularization term) and check the result

In [65]:
# Evaluate Cost J (with regularization term)
J=0
parameters=[]

# YOUR CODE ..................................
# call cofiCostFunc with lamb=1.5 (without regularization term)


# Aplanar las matrices X y Theta para crear el vector de parámetros
parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Llamar a cofiCostFunc con lamb = 1.5 (con término de regularización)
J, _ = cofiCostFunc(parameters, Y, R, n_users, n_movies, n_features, lamb=1.5)


# YOUR CODE (end)..................................

print('The value of J (with regularization term equal to 1.5) is %0.2f (it should be 31.34)' % J )

The value of J (with regularization term equal to 1.5) is 31.34 (it should be 31.34)


### 2.4 Computing the gradient of J
The **gradient of J** depends on the two types of parameters, i.e. X and Theta. The corresponding equations are:

$$\frac { \partial J }{ \partial { \theta  }_{ k }^{ (j) } } =\sum _{ i:r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) { x }_{ k }^{ (i) } } +\lambda { \theta  }_{ k }^{ (j) }$$

$$\frac { \partial J }{ \partial { x }_{ k }^{ (i) } } =\sum _{ j:r(i,j)=1 }^{  }{ \left( { x }^{ (i) }({ \theta  }^{ (j) })^{ T }-{ y }^{ (i,j) } \right) \theta _{ k }^{ (j) } } +\lambda { x }_{ k }^{ (i) }$$

### Task 10
***
Now, go back and **complete the cofiCostFunc code to compute the gradient of J**. Remember to use vectorized operations instead of using for loops.

Note that the outputs of cofiCostFunc are the cost J (scalar value) and the gradient, again a **flattened vector of the corresponding gradients of X and Theta**:

$${ \left[ \begin{matrix} \frac { \partial J }{ \partial { x }^{ (1) } } , & ... & \frac { \partial J }{ \partial { x }^{ (m) } } , & \frac { \partial J }{ \partial \theta ^{ (1) } } , & ... & \frac { \partial J }{ \partial \theta ^{ (u) } }  \end{matrix} \right]  }_{ (m\cdot n)+(u\cdot n) }$$

After computing both gradients, you should reshape them into a flattened vector called **gradient** that will be returned by the cofiCostFunc.

### 2.5 Checking the gradient of J
For the same dataset of the last poit, you will check the gradient of J computed by your cofiCostFunc

In [66]:
# Check gradients (without regularization term) by running the next function
checkCostFunction(cofiCostFunc,0)

The above two columns you get should be very similar.
(Left - Your Numerical Gradient, Right - Analytical Gradient)

(-2.1671222630903486, -2.1671222630887312)
(-1.869123836848452, -1.8691238368475827)
(-0.9300152790459038, -0.9300152790452532)
(-1.5841439076647035, -1.5841439076659005)
(-1.188446170479196, -1.1884461704788276)
(-0.8390474059893904, -0.8390474059899131)
(-1.0626437899041719, -1.0626437899038597)
(-0.5930092499539796, -0.5930092499513944)
(-0.32522667815726436, -0.3252266781573216)
(0.6138219467599626, 0.6138219467592629)
(0.7703441066952976, 0.7703441066988151)
(0.6421432611958267, 0.6421432611923157)
(-1.461896466650181, -1.461896466651514)
(-0.696525969954287, -0.6965259699584189)
(-2.2256863179181963, -2.2256863179217206)
(0.013073552138642697, 0.013073552137715801)
(0.16204940705311088, 0.16204940705401016)
(-0.03661503070073735, -0.03661503070342405)
(-0.03125837838791057, -0.03125837839038152)
(0.23852381616062956, 0.23852381616025845)
(-0.3482475325489176, -0.34

In [67]:
# Check gradients (with regularization term) by running the next function
checkCostFunction(cofiCostFunc,1.5)

The above two columns you get should be very similar.
(Left - Your Numerical Gradient, Right - Analytical Gradient)

(-0.23208114839690097, -0.23208114839975652)
(-0.8143033805252387, -0.814303380516763)
(-0.6702195264329802, -0.6702195264386588)
(0.30772342345386505, 0.3077234234538109)
(0.9216747475093712, 0.9216747475038201)
(0.02001694605180404, 0.020016946058665774)
(1.358054141453735, 1.3580541414523268)
(0.7684705299970673, 0.768470529998766)
(0.3163403082817595, 0.3163403082795445)
(1.1342937174996237, 1.1342937174996144)
(1.5121186213251647, 1.5121186213235793)
(1.3885875112418233, 1.3885875112427453)
(0.0012647084890460292, 0.0012647084900785366)
(0.9062907786239194, 0.9062907786332649)
(1.0324446249443398, 1.032444624946855)
(1.077624754404205, 1.0776247544005022)
(1.1175909060634481, 1.1175909060637979)
(1.048878724789759, 1.0488787247897)
(0.78598463298718, 0.7859846329835674)
(1.4007813719052464, 1.400781371902804)
(0.755970458694577, 0.7559704586916994)
(-0.4680006879498

## 3 Learning and recommendation
Finally, you will use your cofiCostFun to make predictions using the initial Movielens dataset. Part of the python code you need is already written in the next cells. You only have to complete those lines that are explicitly required.

### Task 11
***
Load the matrix Y and R computed in the first notebook.

In [68]:
# Task: load matrix Y and R
# YOUR CODE ..................................

# Ruta al archivo 'u.data' en tu carpeta de trabajo de Jupyter
file_path_ratings = 'u.data'

# Leer el archivo 'u.data' y crear las matrices "Y" y "R"
ratings_data = np.genfromtxt(file_path_ratings, delimiter='\t', dtype=int)
num_users = np.max(ratings_data[:, 0])
num_movies = np.max(ratings_data[:, 1])



dim_Y = (num_movies, num_users)
dim_R = (num_movies, num_users)

# Supongamos que estas son las matrices cargadas
Y = np.load('matriz_Y.npy')
R = np.load('matriz_R.npy')

# Verificar las dimensiones de las matrices Y y R
if Y.shape == dim_Y and R.shape == dim_R:
    print("Las dimensiones son correctas:")
    print(f"Dimensiones de Y: {Y.shape} - Esperadas: {dim_Y}")
    print(f"Dimensiones de R: {R.shape} - Esperadas: {dim_R}")
else:
    print("Error en las dimensiones:")
    print(f"Dimensiones de Y: {Y.shape} - Esperadas: {dim_Y}")
    print(f"Dimensiones de R: {R.shape} - Esperadas: {dim_R}")
    
# Imprimir una muestra de las matrices Y y R
print("Matriz Y:")
print(Y[:5, :5])  # Imprime las primeras 5 filas y 5 columnas de Y

print("\nMatriz R:")
print(R[:5, :5])  # Imprime las primeras 5 filas y 5 columnas de R




Las dimensiones son correctas:
Dimensiones de Y: (1682, 943) - Esperadas: (1682, 943)
Dimensiones de R: (1682, 943) - Esperadas: (1682, 943)
Matriz Y:
[[5. 4. 0. 0. 4.]
 [3. 0. 0. 0. 3.]
 [4. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]]

Matriz R:
[[1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


### Task 12
***
* Get the number of users and movies and assign the corresponding variables n_users, n_movies.
* Set the initial parameters (Theta, X) with random values.
* Fold X and Theta into the variable initial_parameters.

In [69]:
# Set the number of features
n_features = 100

In [77]:
# YOUR CODE ..................................
# Assuming Y and R have already been loaded
n_movies, n_users = Y.shape

# Number of features
n_features = 100

# Initialize X and Theta with random values
X = np.random.rand(n_movies, n_features)
Theta = np.random.rand(n_users, n_features)

# Unroll parameters into a single array
initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])



Now, we set the rest of the parameters and minimize the function

In [80]:
# Set the regularization parameter
lamb = 10

# Define a function to be minimized
def cofiCostFunc_minimize(parameters):
    return cofiCostFunc(parameters,Y, R, n_users, n_movies, n_features,lamb)

# Set the number of iteations
max_iter=200

In [81]:
print(Y.shape)
print(R.shape)



# Minimize the function using minimize from the package scipy.optimize and get the optimized parameters
parameters = (minimize(cofiCostFunc_minimize, initial_parameters, method="CG", jac=True,
                       options={'maxiter': max_iter, "disp": True})).x


(1682, 943)
(1682, 943)
         Current function value: 66580.046998
         Iterations: 200
         Function evaluations: 293
         Gradient evaluations: 293


### Task 13
***
Get the matrix of predictions P


In [82]:
# YOUR CODE ..................................

# Reshape the optimized parameters back into X and Theta matrices
X_optimized = parameters[:n_movies * n_features].reshape(n_movies, n_features)
Theta_optimized = parameters[n_movies * n_features:].reshape(n_users, n_features)
print(X_optimized.shape)
print(Theta_optimized.shape)

# Now, you can use X_optimized and Theta_optimized for making predictions or further analysis

# Calcular la matriz de predicciones P
P = np.dot(X_optimized, Theta_optimized.T)
print(P.shape)
# P ahora contiene las predicciones de calificaciones para cada par de película y usuario

(1682, 100)
(943, 100)
(1682, 943)


### Task 14
***
Show the titles of the top-5 predictions for the first user u=0, for those films user u did not watch: r(i,u)=0 (they will be the top-5 recommendations)

#### Tips
* You can import movies' titles using Pandas (see the first notebook)


In [103]:
# YOUR CODE ..................................

# import pandas
from pandas import read_table
# read csv file
items = read_table('u.item',header=None,sep='|',encoding='ISO-8859-1')
# remove collumns 2-24
items.drop(range(2,24),axis=1, inplace=True)
# name the columns
items.columns = ['itemid','title']
# show the first 5 rows of the dataframe
items.head()

# Continuando con tus variables existentes

# Identificar películas no vistas por el usuario u=0
not_watched_by_user = np.where(R[0, :] == 0)[0] # Obtiene los índices de las películas no vistas
predictions_for_user = P[not_watched_by_user, 0]
top_5_indices_not_watched = np.argsort(predictions_for_user)[-5:]
top_5_movie_indices = not_watched_by_user[top_5_indices_not_watched]
top_5_movie_titles = items.iloc[top_5_movie_indices]['title']

# Mostrar los títulos y las predicciones correspondientes
print(top_5_movie_titles)
print(predictions_for_user[top_5_indices_not_watched])


np.max(P[:,0])

# Mostrar los títulos de las 5 mejores predicciones
#print(top_5_movie_titles)






49               Star Wars (1977)
99                   Fargo (1996)
168    Wrong Trousers, The (1993)
407         Close Shave, A (1995)
284         Secrets & Lies (1996)
Name: title, dtype: object
[5.03385052 5.06223251 5.1203045  5.20062706 5.51531602]


5.515316019193881